# **CONVOLUTIONAL NEURAL NETWORK (CNN)**

## Problem

Write an MNIST classifier that trains to 99% accuracy or above.
You should stop training once you reach that level of accuracy.

Hint: It should succeed in less than 10 epochs.

## Initialize

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from matplotlib import pyplot as plt

In [ ]:
# set random seeds
np.random.seed(0)
tf.random.set_seed(0)

# show figures inline
%matplotlib inline

## Dataset

Load the MNIST dataset from `tf.keras`.

**Note**:
- For `Conv2D` we need images to have three dimension.
- Use `np.expand_dims` along `axis=-1` (last axis).

In [ ]:
# load dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# expand dims
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# check shapes
print("shapes:")
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
# check one image
idx = 2

# plot
plt.figure()
plt.title(f"Size: {x_train[idx].shape}\nLabel: {y_train[idx]}")
plt.axis('off')
plt.imshow(x_train[idx].squeeze())
plt.show()

**Normalization**
- It allows the NN model to learn all optimal parameters more quickly.

In [ ]:
# normalize
x_train = x_train / 255.
x_test = x_test / 255.

## Model

**Note**

- Use `'softmax'` activation function in the output layer since the problem is multi-class classification.
- Use `'sparse_categorical_crossentropy'` loss since labels are of ordinal category.
- Alternatively, you can convert labels to one-hot-encoding, then use `'categorical_crossentropy'` loss.

In [ ]:
# input layer
input_tensor = Input(shape=x_train.shape[1:])

# convolution layers
x = Conv2D(16, 3, activation='relu')(input_tensor)
x = MaxPool2D()(x)

# fully-connected layers
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# output layer with 'softmax' activation function
output_tensor = Dense(10, activation='softmax')(x)

# model
model = tf.keras.Model(input_tensor, output_tensor)

# compile with 'sparse_categorical_crossentropy' loss
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['acc']
)

# model summary
model.summary()

## Callback

- You can use [pre-defined callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks).
- Or, you can define custom callback to have more control over what happens during the training or prediction.
[See more](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback).

**Note**:
- Use `log.get()` to get desired metric to monitor.
- The parameter name in `log.get()` should match the metric defined in `model.compile()`.

In [ ]:
class MonitorAccuracy(tf.keras.callbacks.Callback):
    """
    a custom class of callback
    to check accuracy after end of each epoch, and
    to stop training when a certain level of accuracy is reached
    """

    def __init__(self, stop_accuracy=0.99):
        # initiate
        super(MonitorAccuracy, self).__init__()
        self.stop_accuracy = stop_accuracy

    def on_epoch_end(self, epoch, logs=None):
        # at the end of the epoch, print loss and accuracy
        print(f"Epoch {epoch+1} - loss: {logs.get('loss'):.4f} - acc: {logs.get('acc'):.4f}")

        # if accuracy is greater than the given 'stop_accuracy':
        if logs.get('acc') > self.stop_accuracy:
            # print the termination message
            print(f"\nAccuracy reached to {self.stop_accuracy}. So, cancelling training...")
            # stop training
            self.model.stop_training = True


monitor_acc = MonitorAccuracy(0.99)

## Training

In [ ]:
# train the model and save the history
hist = model.fit(
    x_train, y_train,
    epochs=10,
    verbose=0,
    callbacks=[monitor_acc]
)

In [ ]:
# plot the loss and accuracy
fig = plt.figure()
ax1 = fig.gca()

ax2 = ax1.twinx()
ax1.plot(hist.history['acc'], label='Accuracy', color='r')
ax2.plot(hist.history['loss'], label='Loss', color='b')

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy', color='r')
ax2.set_ylabel('Loss', color='b')

plt.show()

## Evauation

In [ ]:
# test
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

**Confusion matrix**

In [ ]:
# confusion matrix
conf_mat = confusion_matrix(y_test, y_pred, normalize='true')

In [ ]:
# figure for displaying the confusion matrix
fig = plt.figure(figsize=(6, 6))
ax = fig.gca()

# display the confution matrix
cax = ax.matshow(conf_mat, cmap='Blues')

# show the values
for (i, j), z in np.ndenumerate(conf_mat):
    text_color = 'w' if i == j else 'k'
    if z < 0.005:
        continue
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center', c=text_color)

# title and axis labels
plt.title('Confusion matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# show class names
labels = list(range(10))
plt.xticks(labels, labels)
plt.yticks(labels, labels)
ax.tick_params(axis='both', which='both', length=0)

# show grid lines
ax.set_xticks(np.arange(-.5, 10, 1), minor=True)
ax.set_yticks(np.arange(-.5, 10, 1), minor=True)
ax.grid(which='minor', color='k', linestyle='-', linewidth=1)

plt.show()

**Other metrics**

In [ ]:
# calculate precision, recall, and f1 score
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

In [ ]:
# pandas data frame for storing the metrics
df = pd.DataFrame({
        'Precision': precision,
        'Recall': recall,
        'F1-score': f1
})

# calculate the mean
df.loc['Mean'] = df.mean()

# display
df.T